#Laboratorio 9

## Task 1 - Teoría

Responda las siguientes preguntas de forma clara y concisa, pueden subir un PDF bien dentro del mismo Jupyter Notebook.

1. Diga cual es la diferencia entre Modelos de Markov y Hidden Markov Models

En el caso de los modelos de markov, estos son modelos estocásticos donde se tiene la condición de que el sistema actual depende únicamente del estado anterior, lo cuál es la propiedad de markov. En estos modelos los estados son completamente observables, no hay información oculta y las transiciones entre los estados tienen probabilidad fijas generalmente relacionadas a una matriz de probabilidades. En el caso de los Hidden Markov models, estos son una extensión de los modelos de markov donde hay mayor ambiguedad debido a que el ambiente no es completamente observable, hay estados ocultos. Ahora bien, los estados ocultos no son indetectables ya que de igual manera estos "emiten señales" indirectamente llamadas emisiones observables. Aunque sea más complejo es útil para poder modelar situaciones más reales donde no se puede predecir completamente sobre el ambiente y es necesario hacer inferencias debido a estados ocultos. En los HMM se ven las emisiones probocadas por los estados ocultos. Importante mencionar, que aquí se está hablando de solo 1 cadena.

2. Investigue qué son los factorial HMM (Hidden Markov Models)

Estos son una extensión de los HMM que tiene como proposito ayudar a modelar sistemas más complejos. Sistmas donde distintios procesos independientes pueden influir en las observaciones/emisiones. En este caso no solo tenemos una cadena oculta, sin varios cadenas que influyen en las observaciones. A diferencia del modelo original de HMM que toma en consideración solo 1 cadena para una observación. Este modelos es útil en situaciones complejas cuando un evento puede estar afectado por varas cadenas de markov ocultas. Por ejemplo en el diagnóstico médico. Si se ven los síntomas de un paciente, las enfermedades causantes de la condición del paciente pueden ser varias. Estas enfermedades son independientes unas de otras y puede que los síntomas puedan ser por culpa de alguna de ellas o de ambas.

3. Especifique en sus propias palabras el algoritmo Forward Backward para HMM

Por lo que entiendo el Algoritmo Forward backward para HMM sirve principalmente para determinar las probabilidades de estar en cada uno de los estados ocultos tomando en cuenta tanto el presente como el futuro, dada una secuencia completa de observaciones en un tiempo. El algoritmo tiene 2 fases principales: el "Forward" y el "Backward". Al avanzar en las observaciones en el tiempo, se calculan las probabilidades de estar en un estado específico tomando solo en cuenta las observaciones que se tienen hasta ese momento. Esta probabilidad parcial se cálcula para los posibles estados ocultos a medida que avanzamos en las observaciones. Luego es cuando viene la segunda fase "Backward", que como indica el normal lo que se hace es revisar las observaciones hacia atras (generalmente de forma recursiva). De tal forma que ahora se cálcula como las observaciones futuras afectan a la probabilidad de estar en un estado. El backward no modifica directamente la probabilidad sino que ayuda a obtener un factor de ajuste para las probabilidades previamente calculadas con el backward. Luego de hacer estos 2 algoritmos se hace algo llamado "Probabilidad suavizada" que es la combinación de ambos cálculos para obtener las probabilidades de estados ocultos dadas las observaciones.

4. En el algoritmo de Forward Backward, por qué es necesario el paso de backward (puede escribir ejemplos o casos para responder esta pregunta)

Si solo se utilizara el forward no quiere decir que fuera incorrecto, pero si sería un tanto incompleto. Con el forward se van calculando las probabilidades con las observaciones sin tomar en cuenta el futuro. Por lo que las estimasiones no son completas, porque en cada paso no se sabe lo que realmente pasó al final y solo se toman en cuenta las observaciones hasta ese momento (observaciones pasadas). Por ejemplo, si estamos hablando del clima, siendo los estados ocultos el clima real y las observaciones el comportamiento de la gente se puede ver de la siguiente forma:

Observaciones:
* día 1:"las personas llevan sombrilla"
* día 2:"las personas llevan lentes de sol"

Si calculamos la probabilidad de haber estado en un día con lluvia (estado oculto) en el día 1, la probabilidad sería alta si solo tomamos en cuenta la observación hasta el día 1. Pero en el siguiente día la observación indica que hay personas usando lentes de sol, probablemente ese día fue soleado. Con la ayuda de la parte de backward lo que se busca al final es tomar en cuenta el futuro para poder alterar las probabilidades ya calculadas. En el contexto de la situación planteada si luego de un día probablemente "lluvioso" el día es "soleado" (o probablemente soleado), debería hacerse un ajuste de la probabilidad del día lluvioso ya que en el futuro hay observaciones que podrían implicar que la probabilidad de haber alcanzado el estado oculto "lluvia" sería menor. Este ajuste se hace por medio de las probabilidades definidas en una matriz de transcición claramente.

## Task 2 - Algoritmo Forward Backward en HMM

El algoritmo forward-backward se basa en una forma de programación dinámica.Para este task deberán investigar sobre el mismo. En este ejercicio estamos ante un modelo meteorológico representado por un Modelo Oculto de Markov (HMM) con dos estados: "Soleado" y "Lluvioso". Queremos predecir el tiempo en un día determinado basándonos en las observaciones de si el día anterior estuvo soleado o lluvioso. Para ello considere el siguiente código como una guía:

Con esto, considere lo siguiente:
1. Defina los parámetros del modelo oculto de Markov (HMM), incluidos estados, observaciones,
probabilidades iniciales, probabilidades de transición y probabilidades de emisión.
2. Cree una instancia de la clase HMM con los parámetros definidos.
3. Genere una secuencia de observaciones utilizando el método generate_sequence. Este paso es opcional
pero útil para realizar pruebas.
4. Utilice el método forward para calcular las probabilidades directas, que representan la probabilidad de
observar una secuencia particular de observaciones hasta un paso de tiempo específico y estar en un estado
particular en ese paso de tiempo.
5. Utilice el método backward para calcular las probabilidades hacia atrás, que representan la probabilidad de
observar la secuencia restante desde un paso de tiempo específico hasta el final de la secuencia, dado que
el sistema se encuentra en un estado particular en ese paso de tiempo.
6. Utilice el método compute_state_probabilities para calcular las probabilidades de estar en cada
estado en cada paso de tiempo dada la secuencia observada. Este paso implica combinar las probabilidades
hacia adelante y hacia atrás.
7. Imprima o analice las probabilidades calculadas según sea necesario.

In [7]:
import random
import numpy as np

class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        """Inicializar parámetros de HMM"""
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob

    def generate_sequence(self, length):
        """Generar una secuencia de observaciones basada en el HMM."""
        # Elegir el estado inicial basado en initial_prob
        current_state = random.choices(self.states,
                                      weights=[self.initial_prob[state] for state in self.states],
                                      k=1)[0]

        sequence = []
        for _ in range(length):
            # Generar observación basada en el estado actual
            observation = random.choices(self.observations,
                                        weights=[self.emission_prob[current_state][obs] for obs in self.observations],
                                        k=1)[0]
            sequence.append(observation)

            # Transición a un nuevo estado
            current_state = random.choices(self.states,
                                          weights=[self.transition_prob[current_state][next_state] for next_state in self.states],
                                          k=1)[0]

        return sequence

    def forward(self, observations):
        """Implementar el paso hacia adelante del algoritmo hacia adelante."""
        # Inicializar matriz de probabilidades forward
        forward = {}
        T = len(observations)

        # Inicialización (t=0)
        for state in self.states:
            forward[(state, 0)] = self.initial_prob[state] * self.emission_prob[state][observations[0]]

        # Recursión
        for t in range(1, T):
            for state_j in self.states:
                forward[(state_j, t)] = 0
                for state_i in self.states:
                    # Sumar probabilidades de llegar a state_j desde cualquier estado previo
                    forward[(state_j, t)] += forward[(state_i, t-1)] * self.transition_prob[state_i][state_j]
                # Multiplicar por la probabilidad de emisión
                forward[(state_j, t)] *= self.emission_prob[state_j][observations[t]]

        return forward

    def backward(self, observations):
        """Implementar el paso hacia atrás del algoritmo hacia adelante."""
        # Inicializar matriz de probabilidades backward
        backward = {}
        T = len(observations)

        # Inicialización (t=T-1)
        for state in self.states:
            backward[(state, T-1)] = 1.0

        # Recursión
        for t in range(T-2, -1, -1):
            for state_i in self.states:
                backward[(state_i, t)] = 0
                for state_j in self.states:
                    # Sumar probabilidades de todos los caminos futuros
                    backward[(state_i, t)] += self.transition_prob[state_i][state_j] * \
                                            self.emission_prob[state_j][observations[t+1]] * \
                                            backward[(state_j, t+1)]

        return backward

    def compute_state_probabilities(self, observations):
        """Combine probabilidades hacia adelante y hacia atrás para calcular probabilidades de estado"""
        forward_probs = self.forward(observations)
        backward_probs = self.backward(observations)
        T = len(observations)
        state_probs = {}
        for t in range(T):
            # denominador para normalizar
            denom = sum(
                forward_probs[(s, t)] * backward_probs[(s, t)]
                for s in self.states
            )
            for s in self.states:
                numer = forward_probs[(s, t)] * backward_probs[(s, t)]
                state_probs[(s, t)] = numer / denom if denom > 0 else 0.0
        return state_probs


In [8]:
# Uso y datos
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Sunny', 'Rainy']
initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}
transition_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}}
emission_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}}

hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

# Generar una secuencia de observaciones.
obs_sequence = hmm.generate_sequence(5)
print("Secuencia Generada:", obs_sequence)

# Calculo de probabilidades forward
forward_probs = hmm.forward(observations)
print("\nProbabilidades Forward:")
print(forward_probs)

# Calculo de probabilidades backward
backward_probs = hmm.backward(observations)
print("\nProbabilidades Backward:")
print(backward_probs)

# Calcular probabilidades de estado
state_probs = hmm.compute_state_probabilities(observations)
print("\nProbabilidades de Estado:")
print(state_probs)

Secuencia Generada: ['Sunny', 'Rainy', 'Sunny', 'Sunny', 'Rainy']

Probabilidades Forward:
{('Sunny', 0): 0.4, ('Rainy', 0): 0.15, ('Sunny', 1): 0.30400000000000005, ('Rainy', 1): 0.051000000000000004, ('Sunny', 2): 0.05272000000000002, ('Rainy', 2): 0.06398000000000001}

Probabilidades Backward:
{('Sunny', 2): 1.0, ('Rainy', 2): 1.0, ('Sunny', 1): 0.30000000000000004, ('Rainy', 1): 0.5, ('Sunny', 0): 0.22200000000000006, ('Rainy', 0): 0.18600000000000003}

Probabilidades de Estado:
{('Sunny', 0): 0.7609254498714653, ('Rainy', 0): 0.23907455012853465, ('Sunny', 1): 0.7814910025706941, ('Rainy', 1): 0.21850899742930588, ('Sunny', 2): 0.45175664095972584, ('Rainy', 2): 0.5482433590402742}
